Recognizing Shapes in Objects with OpenCV
=========================================


Import libraries.

In [1]:
import cv2
import numpy as np
import os # Contains os.path python method/function

Image filenames.

In [2]:
#Tuple data structure is used to store file names

files = (   
    'breakfast.jpg',
    'landscape-house.jpg',
    'building.jpg'
)

# Try the format image_file = r'..\..\..\images\xxx.jpg' if any problems with reading an image.

print("Set x[0 to 2] in files[x] in order read desired image")
image_file = os.path.join("images",files[0])


Set x[0 to 2] in files[x] in order read desired image


Define a function for viewing images.

In [3]:
def imageviewer(f):
    cv2.imshow('view',f)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

Read an image from file.

In [4]:
# Check if image can be read.
b = cv2.haveImageReader(image_file)
print(b)

# Read and show image
i=cv2.imread(image_file)
imageviewer(i)

True


Inspect image contents

In [5]:
print(i.shape)
print(i[0,0,:])

(1320, 1979, 3)
[  0  86 176]


Gray-scale

In [6]:
# Need to convert image to gray scale as most tools wouldn't work well with RGB images
gray_img = cv2.cvtColor(i,cv2.COLOR_BGR2GRAY)
imageviewer(gray_img)
print(gray_img[0,0])
print(gray_img.shape)

103
(1320, 1979)


X gradient

In [7]:
# Differentitate with Sobel function in OpenCV which will be useful for detection of edges
# Look for brightness increases in the x-direction with following code.
x_gradient = cv2.Sobel(gray_img,cv2.CV_64F,1,0)

# Need to get postive values
x_gradient_abs = np.absolute(x_gradient)

# Divide by maximum value/brightness to normalize to between 0 and 1.
imageviewer(x_gradient_abs/np.max(x_gradient_abs))

Y gradient

In [8]:
# Look for brightness increases in the y-direction with following code.
y_gradient = cv2.Sobel(gray_img,cv2.CV_64F,0,1)

# Need to get postive values
y_gradient_abs = np.absolute(y_gradient)

# Divide by maximum value/brightness to normalize to between 0 and 1.
imageviewer(y_gradient_abs/np.max(y_gradient_abs))

Magnitude of gradient vector

In [9]:
# Sum x and y gradients
sum = np.sqrt(x_gradient**2+y_gradient**2)
imageviewer(sum/(np.max(sum)))

Canny edge detection

In [10]:
edges = cv2.Canny(gray_img,50,150) #Adjust integer parameters to either reduce or increase number of edges show.
                  
imageviewer(edges)

Hough transform for lines

In [11]:
# Detecting lines using Hough transforms where x and y coordinates are replaced by rho and theta so thet can't be infinite gradients
# rho => y pixels and theta => x pixels

lines = cv2.HoughLinesP(edges,rho=1,theta=1.*np.pi/180.0,threshold=20,minLineLength=25,maxLineGap=5)
i_lines = i.copy()

for l in lines:
    x1,y1,x2,y2 = l[0]
    cv2.line(i_lines,(x1,y1),(x2,y2),(0,0,255),thickness=3)
    
imageviewer(i_lines)


Hough transform for circles

In [12]:
# References: 1) https://theailearner.com/tag/cv2-houghcircles/
#             2) https://www.delftstack.com/howto/python/opencv-draw-circle/

# Apply Hough transform to greyscale image
circles = cv2.HoughCircles(gray_img,cv2.HOUGH_GRADIENT,dp=2,minDist=20,param1=150,param2=40,minRadius=15,maxRadius=25)

circles_copy = circles.copy()
# convert the (x, y) coordinates and radius of the circles to integers
circles = np.uint16(np.around(circles_copy))


# Draw the circles
for x in circles[0,:]:
    # draw the outer circle
    cv2.circle(i,(x[0],x[1]),x[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(i,(x[0],x[1]),2,(0,0,255),3)

print('Show circles')
imageviewer(i)



'''
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")


detected_circles = np.uint16(np.around(circles))
detected_circles = circles.copy()

for x, y, r in circles[0,:]:
        cv2.circle(detected_circles, x, y,r, (0, 0, 255), 3)
        #cv.circle(detected_circles, (x, y), 2, (255, 0, 0), 3)


imageviewer(circles)
'''

Show circles


'\nif circles is not None:\n    # convert the (x, y) coordinates and radius of the circles to integers\n    circles = np.round(circles[0, :]).astype("int")\n\n\ndetected_circles = np.uint16(np.around(circles))\ndetected_circles = circles.copy()\n\nfor x, y, r in circles[0,:]:\n        cv2.circle(detected_circles, x, y,r, (0, 0, 255), 3)\n        #cv.circle(detected_circles, (x, y), 2, (255, 0, 0), 3)\n\n\nimageviewer(circles)\n'

Blur the image first

Circle detection on blurred image